### Importar Bibliotecas

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyodbc
import sklearn
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import model_selection
import seaborn as sn
from sklearn.preprocessing import RobustScaler
from scipy import stats
from sklearn.decomposition import PCA
from numpy import percentile
from sklearn.model_selection import train_test_split

### Configuração SQL Server e importação dataset treino

In [2]:
#### Configurar ligação ao SQL Server

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=.;' 
                      'Database=Datawarehouse;'
                      'UID=ole;'
                      'PWD=12qwaszx*')


In [3]:
#### Importar Dataset para o PYNB

df = pd.read_sql_query('SELECT * FROM  ML_DATASET_FINAL',conn)
df = df.drop(["RESERVATION_COD"],axis = 1)

In [4]:
df_orig = df 

### Data Pre Processing

#### Tratar Missing Values

In [5]:
#df = df_orig
#df = df.dropna()
df = df.fillna(df.median())

#### Transformação Logaritmica para corrigir enviesamento

In [6]:
#df["ABT"] = np.log1p(df["ABT"])
#df["LOR"] = np.log1p(df["LOR"])
#df["AVG_OCP_GLOBAL_RATE"] = np.log1p(df["AVG_OCP_GLOBAL_RATE"])
#df["AVG_OCP_ZONE_RATE"] = np.log1p(df["AVG_OCP_ZONE_RATE"])
#df["AVG_OCP_STATION_RATE"] = np.log1p(df["AVG_OCP_STATION_RATE"])
#df["N_RESERVAS_ANTERIORES"] = np.log1p(df["N_RESERVAS_ANTERIORES"])
#df["N_CARROS_DISPONIVEIS"] = np.log1p(df["N_CARROS_DISPONIVEIS"])
#df["AVG_OCP_RATE_RESERVA"] = np.log1p(df["AVG_OCP_RATE_RESERVA"])
#df["PRICE_DAY"] = np.log1p(df["PRICE_DAY"])

#### Gerar novas colunas de Data para as 3 Datas (Reserva, Out, In)

In [7]:
################ RESERVATION DATE ##########################

## Converter para datetime
df['RESERVATION_DATE'] = pd.to_datetime(df['RESERVATION_DATE'], 
 format = '%Y%m%d', 
 errors = 'coerce')

##Gerar as várias colunas
df['R_Year'] = df['RESERVATION_DATE'].dt.year
df['R_Month'] = df['RESERVATION_DATE'].dt.month
#df['R_Week'] = df['RESERVATION_DATE'].dt.week
#df['R_Day'] = df['RESERVATION_DATE'].dt.day
df['R_DayofWeek'] = df['RESERVATION_DATE'].dt.dayofweek

#Droppar a coluna original
df = df.drop("RESERVATION_DATE",1)

############### OUT DATE ##########################

# Converter para datetime
df['OUT_DATE'] = pd.to_datetime(df['OUT_DATE'], 
 format = '%Y%m%d', 
 errors = 'coerce')

#Gerar as várias colunas
df['O_Year'] = df['OUT_DATE'].dt.year
df['O_Month'] = df['OUT_DATE'].dt.month
#df['O_Week'] = df['OUT_DATE'].dt.week
#df['O_Day'] = df['OUT_DATE'].dt.day
df['O_DayofWeek'] = df['OUT_DATE'].dt.dayofweek

#Droppar a coluna original
df = df.drop("OUT_DATE",1)

############### IN DATE ########################

df['IN_DATE'] = pd.to_datetime(df['IN_DATE'], 
 format = '%Y%m%d', 
 errors = 'coerce')

#Gerar as várias colunas
df['I_Year'] = df['IN_DATE'].dt.year
df['I_Month'] = df['IN_DATE'].dt.month
#df['I_Week'] = df['IN_DATE'].dt.week
#df['I_Day'] = df['IN_DATE'].dt.day
df['I_DayofWeek'] = df['IN_DATE'].dt.dayofweek

#Droppar a coluna original
df = df.drop("IN_DATE",1)

#### One-hot encoding (criação de dummy attributes) para as Features Categóricas

In [8]:
df = pd.get_dummies(df, columns=['VEHICLE_GROUP_COD'], prefix = ['VG'])
df = pd.get_dummies(df, columns=['VECHICLE_CLUSTER'], prefix = ['VC'])
df = pd.get_dummies(df, columns=['VEHICLE_SEGMENT'], prefix = ['VS'])

df = pd.get_dummies(df, columns=['IN_STATION'], prefix = ['IS'])
df = pd.get_dummies(df, columns=['IN_ZONE'], prefix = ['IZ'])
df = pd.get_dummies(df, columns=['IN_ACTIVITY'], prefix = ['IA'])

df = pd.get_dummies(df, columns=['OUT_STATION'], prefix = ['OS'])
df = pd.get_dummies(df, columns=['OUT_ZONE'], prefix = ['OZ'])
df = pd.get_dummies(df, columns=['OUT_ACTIVITY'], prefix = ['OA'])


df = pd.get_dummies(df, columns=['CLIENT_TYPE'], prefix = ['CT'])
df = pd.get_dummies(df, columns=['CLIENT_SUBTYPE'], prefix = ['CST'])
df = pd.get_dummies(df, columns=['VECHICLE_TYPE'], prefix = ['VT'])

# One Hot Encoding dos atributos da data

df = pd.get_dummies(df, columns=['R_Year'], prefix = ['RY'])
df = pd.get_dummies(df, columns=['R_Month'], prefix = ['RM'])
df = pd.get_dummies(df, columns=['R_DayofWeek'], prefix = ['RD'])

df = pd.get_dummies(df, columns=['I_Year'], prefix = ['IY'])
df = pd.get_dummies(df, columns=['I_Month'], prefix = ['IM'])
df = pd.get_dummies(df, columns=['I_DayofWeek'], prefix = ['ID'])

df = pd.get_dummies(df, columns=['O_Year'], prefix = ['OY'])
df = pd.get_dummies(df, columns=['O_Month'], prefix = ['OM'])
df = pd.get_dummies(df, columns=['O_DayofWeek'], prefix = ['OD'])

# Replace de square brackets no nome das colunas
df.columns=df.columns.str.replace('[','')
df.columns=df.columns.str.replace(']','')

### Feature Selection

#### 1) Variance Threshold - Remover features com baixa variância

In [29]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit(X)
X_transformed_VT = X.loc[:, sel.get_support()]
X_transformed_VT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275711 entries, 0 to 1275710
Data columns (total 28 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   ABT                           1275711 non-null  int64  
 1   LOR                           1275711 non-null  int64  
 2   N_RESERVAS_ANTERIORES         1275711 non-null  float64
 3   N_CARROS_DISPONIVEIS          1275711 non-null  float64
 4   AVG_OCP_RATE_RESERVA          1275711 non-null  float64
 5   VC_Compactos                  1275711 non-null  uint8  
 6   VC_Mini/Económicos            1275711 non-null  uint8  
 7   VS_Compactos                  1275711 non-null  uint8  
 8   VS_M+E                        1275711 non-null  uint8  
 9   IS_RAC - Aeroporto Lisboa     1275711 non-null  uint8  
 10  IZ_Lisboa                     1275711 non-null  uint8  
 11  IZ_Porto                      1275711 non-null  uint8  
 12  OS_RAC - Aeroporto Lisboa   

#### 2) Univariate Feature Selection 

In [30]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

# Create and fit selector
#selector = SelectKBest(f_regression, k=5)
selector = SelectKBest(mutual_info_regression, k=30)

selector.fit(X, y)

# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
X_transformed_UFS = X.iloc[:,cols]
X_transformed_UFS.info()

MemoryError: Unable to allocate 5.31 GiB for an array with shape (559, 1275711) and data type float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275711 entries, 0 to 1275710
Data columns (total 5 columns):
 #   Column              Non-Null Count    Dtype
---  ------              --------------    -----
 0   VC_Mini/Económicos  1275711 non-null  uint8
 1   VC_Premium          1275711 non-null  uint8
 2   VS_7&9L             1275711 non-null  uint8
 3   VS_Automáticos      1275711 non-null  uint8
 4   VS_M+E              1275711 non-null  uint8
dtypes: uint8(5)
memory usage: 6.1 MB


#### Feature Scaling e tratamento de Outliers

In [ ]:
#### Tratar Outliers e Scaling 

trans = RobustScaler()
df[["ABT","LOR", "AVG_OCP_GLOBAL_RATE" ,"AVG_OCP_ZONE_RATE","AVG_OCP_STATION_RATE", "N_RESERVAS_ANTERIORES","N_CARROS_DISPONIVEIS","AVG_OCP_RATE_RESERVA"]] = trans.fit_transform(df[["ABT","LOR", "AVG_OCP_GLOBAL_RATE", "AVG_OCP_ZONE_RATE", "AVG_OCP_STATION_RATE", "N_RESERVAS_ANTERIORES","N_CARROS_DISPONIVEIS","AVG_OCP_RATE_RESERVA"]])

# calculate interquartile range
q25 = percentile(df["PRICE_DAY"], 25)
q75 = percentile(df["PRICE_DAY"], 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))

# calculate the outlier cutoff
cut_off = iqr * 1.5
lower = q25 - cut_off
upper = q75 + cut_off
print(upper)
print(lower)

# Apagar outliers
df = df[ (df["PRICE_DAY"] >= lower) &  (df["PRICE_DAY"] <=  upper) ] 

####  Separar Dataframe Teste e Treino

In [9]:
#### Separar Dataset em Feature Attributes (X) e Target Attribute (y) 

X = df.drop(["PRICE_DAY"],axis = 1) # Todas as features
y = df["PRICE_DAY"].copy()

# Normalizar para otimizar fit da rede neuronal
#X = tf.keras.utils.normalize(X, axis=1)

#### Criar Dataset treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction
test = SelectKBest(score_func=f_classif, k=4)
fit = test.fit(X, Y)
# summarize scores
set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:5,:])

#### Reduzir dimensionalidade - PCA

In [ ]:
#pca = PCA(n_components= 0.50)
#X = pca.fit_transform(X)
#pca.explained_variance_ratio_

## Deep Learning Artificial Neural Network Bull Shit Supreme

### Criar o modelo

In [ ]:
model = tf.keras.models.Sequential()  # a basic sequential feed-forward model

### Criar as várias layers da Rede Neuronal

Vamos começar por criar a Input Layer que deve ser igual ao número de features do nosso X_train. Por default usa-se como função de ativação tf.nn.relu.

In [ ]:
model.add(tf.keras.layers.Dense(559, activation=tf.nn.relu))

Depois vamos criar duas Hidden Layers que vão permitir que a rede compreenda melhor as relações não lineares complexas do problema.

In [ ]:
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(75, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(50, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(25, activation=tf.nn.relu))

Por fim temos a nossa Output Layer que contem apenas um node por se tratar de uma regressão

In [ ]:
model.add(tf.keras.layers.Dense(1))

In [ ]:
 # A função de custo vai ser MAE e usamos o Stochastic Gradient Descent como otimizador
model.compile(loss='mean_absolute_error', optimizer='sgd', metrics=['mse']) 

model.fit(X_train, Y_train, epochs=10)  # train the model

x = model.evaluate(X_test, Y_test)  # evaluate the out of sample data with model
print(x)